In [128]:
import csv
import snscrape.modules.twitter as sntwitter
import itertools
import pandas as pd
import json

In [250]:
OUTPUT_FILE = "../data/results.csv"

In [251]:
def save_tweets(query, n_tweets):
    """
    Finds the latest tweets according to a query and saves them in a csv file
    :param query: the query to look for
    :param n_tweets: number of tweets to scrape
    """
    with open(OUTPUT_FILE, 'w', encoding='utf-8', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['id', 'date', 'username', 'content', 'url'])
        
        tweets_gen = sntwitter.TwitterSearchScraper(query).get_items()
        top_tweets = itertools.islice(tweets_gen, n_tweets)
        
        for tweet in top_tweets:
            writer.writerow([tweet.id, tweet.date, tweet.username, tweet.content, tweet.url])
        
        print('Done!')

In [252]:
query = "#foreignaid"

In [253]:
save_tweets(query=query, n_tweets=10)

Done!


/tmp/ipykernel_4694/2365005399.py:15: DeprecatedFeatureWarning: username is deprecated, use user.username instead
  writer.writerow([tweet.id, tweet.date, tweet.username, tweet.content, tweet.url])
/tmp/ipykernel_4694/2365005399.py:15: DeprecatedFeatureWarning: content is deprecated, use rawContent instead
  writer.writerow([tweet.id, tweet.date, tweet.username, tweet.content, tweet.url])


In [254]:
tweets_gen = sntwitter.TwitterSearchScraper(query).get_items()

In [255]:
type(tweet)

snscrape.modules.twitter.Tweet

In [257]:
[tweet for tweet in tweets_gen]

[]

In [258]:
tweet

Tweet(url='https://twitter.com/janelauppen/status/1414919392350064657', date=datetime.datetime(2021, 7, 13, 12, 7, 25, tzinfo=datetime.timezone.utc), rawContent="#foreignaid @BorisJohnson is a fan of population ctrl. what's more he lies like a trooper so 1000s will die cutting #OverseasAid UK always in debt so he spouts more bull by the day.@oxfamgb @ForeignPolicy @foreignaid @ForeignAidWorks @FCDOGovUK @CFR_org rt wing #austerity again", renderedContent="#foreignaid @BorisJohnson is a fan of population ctrl. what's more he lies like a trooper so 1000s will die cutting #OverseasAid UK always in debt so he spouts more bull by the day.@oxfamgb @ForeignPolicy @foreignaid @ForeignAidWorks @FCDOGovUK @CFR_org rt wing #austerity again", id=1414919392350064657, user=User(username='janelauppen', id=104773715, displayname='Jane', rawDescription='Cornwall. Campaigner for social justice,mental health,carers and human rights,austerity only exists to enrich the rich.', renderedDescription='Cornwall

## Advanced

Advanced Search reference: https://github.com/igorbrigadir/twitter-advanced-search

In [195]:
def advancedTweetSearch(query, loc, start, end, lang="en"):
    TWEET_STOR = []
    LOC_STOR = []
    
    scraped_tweets = sntwitter.TwitterSearchScraper(f'{query} geocode:"{loc}"since:{start} until:{end} lang:"{lang}"').get_items()
    
    sliced_scraped_tweets = itertools.islice(scraped_tweets, 1000)
    
    
    for tweet in sliced_scraped_tweets:
        try:
            LOC_STOR.append(json.loads(tweet.json())['place']['country'])
            TWEET_STOR.append(json.loads(tweet.json()))
        except TypeError:
            continue
        
    return TWEET_STOR, LOC_STOR

In [242]:
query = "#usaid OR #usforeignaid OR #foreignaid OR #foreignassistance OR #economy"
start = '2019-01-01'
end = '2023-01-01'
loc = "2.723583, 113.159180, 3500km"

In [243]:
%time tweet_stor, location_details = advancedTweetSearch(query=query, start=start, end=end, loc=loc)

CPU times: user 2.24 s, sys: 14.7 ms, total: 2.25 s
Wall time: 1min 28s


In [244]:
pd.Series(location_details).value_counts()

India                          656
Bangladesh                     123
Pakistan                        82
Sri Lanka                       26
Australia                        5
Republic of the Philippines      3
Nepal                            2
People's Republic of China       2
Maldives                         1
Brunei                           1
Bhutan                           1
dtype: int64

In [245]:
tweet_stor[0]

{'_type': 'snscrape.modules.twitter.Tweet',
 'url': 'https://twitter.com/IslamabadCCI/status/1609112072469504000',
 'date': '2022-12-31T08:59:48+00:00',
 'rawContent': 'Glimpses of visit of ICCI delegation to Navel Headquarter.\n#ICCI @bakhtawarizafar @FaadWaheed #azharulislamzafar #Navy #economy #cooperation #betterment https://t.co/qJWN7IoAIF',
 'renderedContent': 'Glimpses of visit of ICCI delegation to Navel Headquarter.\n#ICCI @bakhtawarizafar @FaadWaheed #azharulislamzafar #Navy #economy #cooperation #betterment https://t.co/qJWN7IoAIF',
 'id': 1609112072469504000,
 'user': {'_type': 'snscrape.modules.twitter.User',
  'username': 'IslamabadCCI',
  'id': 230966713,
  'displayname': 'Islamabad Chamber of Commerce & Industry',
  'rawDescription': '',
  'renderedDescription': '',
  'descriptionLinks': None,
  'verified': False,
  'created': '2010-12-27T06:11:29+00:00',
  'followersCount': 4291,
  'friendsCount': 143,
  'statusesCount': 2818,
  'favouritesCount': 29,
  'listedCount': 

In [246]:
# brazil

loc = "-8.581021,-51.328125, 4500km"

In [247]:
%time tweet_stor, location_details = advancedTweetSearch(query=query, start=start, end=end, loc=loc)

CPU times: user 2.74 s, sys: 15.4 ms, total: 2.75 s
Wall time: 1min 31s


In [248]:
pd.Series(location_details).value_counts()

United States    834
Ghana             87
Mexico            11
Senegal            6
Brazil             6
Liberia            3
Fiji               3
Guatemala          3
Sierra Leone       3
Spain              2
Bahamas            2
Ivory Coast        1
Portugal           1
Morocco            1
dtype: int64

In [249]:
tweet_stor[0]

{'_type': 'snscrape.modules.twitter.Tweet',
 'url': 'https://twitter.com/DanielW247/status/1608990500891291649',
 'date': '2022-12-31T00:56:43+00:00',
 'rawContent': 'At some point we will all be millionaires as the dollar continues to inflate. 1 million in the 70s is worth 7 million now. 😳   #inflation #money #foodforthought #expenses #economy',
 'renderedContent': 'At some point we will all be millionaires as the dollar continues to inflate. 1 million in the 70s is worth 7 million now. 😳   #inflation #money #foodforthought #expenses #economy',
 'id': 1608990500891291649,
 'user': {'_type': 'snscrape.modules.twitter.User',
  'username': 'DanielW247',
  'id': 1607988453173518337,
  'displayname': 'Daniel Williams',
  'rawDescription': 'Life is what you make it. Businessman, car enthusiast, coffee and cigars. 🤘',
  'renderedDescription': 'Life is what you make it. Businessman, car enthusiast, coffee and cigars. 🤘',
  'descriptionLinks': None,
  'verified': False,
  'created': '2022-12-2